# IMPORTING THE LIBRARIES

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

# IMPORTING THE DATASET

In [ ]:
data = pd.read_csv('../input/churn-modelling/Churn_Modelling.csv')

In [ ]:
data.head()

In [ ]:
# independent feature:
X = data.iloc[:, 3:13]

In [ ]:
# dependent feature :
y = data.iloc[:, 13]

In [ ]:
X.head()

In [ ]:
y.head()

In [ ]:
# Create dummy variables :
geography=pd.get_dummies(X["Geography"],drop_first=True)

In [ ]:
gender=pd.get_dummies(X['Gender'],drop_first=True)

In [ ]:
# Concatenate the Data Frames : 
X=pd.concat([X,geography,gender],axis=1)

In [ ]:
X.head()

In [ ]:
# Drop Unnecessary columns :
X=X.drop(['Geography','Gender'],axis=1)

In [ ]:
X.head()

# TRAIN TEST SPLIT

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

# FEATURE SCALING

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
sc = StandardScaler()

In [ ]:
X_train = sc.fit_transform(X_train)

In [ ]:
X_test = sc.transform(X_test)

# PERFORMING HYPERPARAMETER OPTIMIZATION

In [ ]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense, Activation, Embedding, Flatten, LeakyReLU, BatchNormalization, Dropout
from keras.activations import relu, sigmoid

In [ ]:
def create_model(layers, activation):
    model = Sequential()
    for i, nodes in enumerate(layers):
        if i==0: 
            model.add(Dense(nodes,input_dim=X_train.shape[1]))
            model.add(Activation(activation))
            model.add(Dropout(0.3))
        else:
            model.add(Dense(nodes))
            model.add(Activation(activation))
            model.add(Dropout(0.3))
            
    model.add(Dense(units = 1, kernel_initializer= 'glorot_uniform', activation = 'sigmoid')) 
    
    model.compile(optimizer='adam', loss='binary_crossentropy',metrics=['accuracy'])
    return model
model = KerasClassifier(build_fn=create_model, verbose=0)

In [ ]:
layers = [(20), (40, 20), (45, 30, 15)]
activations = ['sigmoid', 'relu']
param_grid = dict(layers=layers, activation=activations, batch_size = [128, 256], epochs=[30])
grid = GridSearchCV(estimator=model, param_grid=param_grid,cv=5)

In [ ]:
grid_result = grid.fit(X_train, y_train)

# MODEL BEST RESULT

In [ ]:
grid_result.best_score_

In [ ]:
grid_result.best_params_

In [ ]:
pred_y = grid.predict(X_test)

In [ ]:
y_pred = (pred_y > 0.5)

In [ ]:
from sklearn.metrics import confusion_matrix , accuracy_score

In [ ]:
cm = confusion_matrix(y_pred , y_test)

In [ ]:
cm

In [ ]:
score=accuracy_score(y_pred,y_test)

In [ ]:
score